In [7]:
!pip install peft
!pip install transformers

In [8]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from transformers import T5Tokenizer
from torchvision import transforms
import torch
import json
import os
import time
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer, GPT2LMHeadModel, GPT2TokenizerFast
from torchvision.models import vit_b_32
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import argparse
from peft import LoraConfig, get_peft_model
from collections import namedtuple
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TEXT_MAX_LENGTH = 463
VIT_HIDDEN_STATE = 768
VIT_SEQ_LENGTH = 49
GPT_N_EMBED = 1024

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"Trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



class DriveVLM(nn.Module):

    def __init__(self, config):

        super().__init__()

        # Make tokenizer and text model
        if config.lm == 'T5':
            self.model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-base')
            hidden_size = self.model.config.d_model
        else:
            self.model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
            hidden_size = GPT_N_EMBED


        # Only if we are perfoming LoRA finetuning on T5
        if config.lora:

            if config.lm == 'T5':
              tm = ['q', 'v']
            else:
              tm = ['c_attn']

            # Create LoRA model
            lora_config = LoraConfig(
                r=config.lora_dim,
                lora_alpha=config.lora_alpha,
                lora_dropout=config.lora_dropout,
                bias='none',
                target_modules=tm
            )
            self.model = get_peft_model(self.model, lora_config)

        # Freeze model weights if needed
        if config.freeze_lm:

            for param in self.model.parameters():
                param.requires_grad = False

        print('Trainable Parameters for LM model:')
        print_trainable_parameters(self.model)

        # Create instance for multi-view processor
        self.mvp = self.MultiViewProcessor(config.gpa_hidden_size, hidden_size, config.lm, freeze=True)

    class MultiViewProcessor(nn.Module):

        def __init__(self, gpa_hidden_size, hidden_size, lm, freeze=False):

            super().__init__()

            # Use ViT for image embeddings
            self.img_model = vit_b_32(weights='DEFAULT')
            self.lm = lm

            # Modal embedding to distinguish between image and text
            self.modal_embeddings = nn.Embedding(2, hidden_size)
            self.modal_embeddings.weight.data.normal_(mean=0.0, std=0.02)

            # If we are freezing the CLIP embeddings
            if freeze:
                for param in self.img_model.parameters():
                    param.requires_grad = False

            # Set matrices based on MIVC paper
            self.w = nn.Linear(in_features=gpa_hidden_size, out_features=1)
            self.Z = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Tanh()
            )
            self.G = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Sigmoid()
            )

            if self.lm != 'T5':
              self.img_projection_layer = nn.Linear(in_features=VIT_HIDDEN_STATE, out_features=hidden_size)


        def gpa(self, img_embeddings):

            """"
            Calculates the gated-pooling attention score for the image embeddings
            :param img_embeddings: (6x768) dimensional
            :return single embedding of size (768,)
            """

            # Get weights for gated pooling attention
            gpa_weights = torch.softmax(self.w(self.Z(img_embeddings) * self.G(img_embeddings)), dim=0  )

            # Take a linear combination of all the image embeddings
            fused_embeddings = torch.sum(gpa_weights * img_embeddings, dim=0)

            return fused_embeddings

        def get_img_embedding(self, imgs):

            N = imgs.shape[0]

            # Process into patches (N x 6 x 49 x H)
            merged_embedding = torch.stack([self.img_model._process_input(img) for img in imgs], dim=0)

            # Concatenate the batch class tokens -> (N, 6, 50, H)
            batch_class_tokens = self.img_model.class_token.expand(merged_embedding.shape[1], -1, -1).repeat(N, 1, 1, 1)
            merged_embedding = torch.cat([batch_class_tokens, merged_embedding], dim=2)

            # Add positional embeddings and remove class token -> (N, 6, 49, H)
            merged_embedding += self.img_model.encoder.pos_embedding.repeat(N, 1, 1, 1)
            merged_embedding = merged_embedding[:, :, 1:]

            # Get merged embedding and reshape to 2D embedding -> (N, 1, 49, H)
            merged_embedding = torch.stack([self.gpa(embedding.flatten(start_dim=1)).reshape(VIT_SEQ_LENGTH,
                                            VIT_HIDDEN_STATE) for embedding in merged_embedding], dim=0)

            # Project to VL dimension -> (1, 49, H) (H is 512 for t5-small, 768 for t5-base)
            if self.lm != 'T5':
              merged_embedding = self.img_projection_layer(merged_embedding)

            # Add modal type embedding to merged embedding
            merged_embedding += self.modal_embeddings(
                torch.ones((1, merged_embedding.shape[1]), dtype=torch.int, device=device))

            return merged_embedding

        def forward(self, text_enc, imgs, text_model):

            # Get the image embeddings (N x 1 x 49 x H)
            imgs_embedding = self.get_img_embedding(imgs)

            # Get the text embeddings (N x S x H)
            text_embeddings = text_model.get_input_embeddings()(text_enc)

            # Add modal embeddings to text
            text_embeddings += self.modal_embeddings(torch.zeros((1, text_embeddings.shape[1]), dtype=torch.int,
                                                                 device=device))

            # Concatenate embeddings -> (1 x S x 512)
            merged_embedding = torch.cat([text_embeddings, imgs_embedding], dim=1)

            return merged_embedding

    def forward(self, text_enc, imgs, labels=None):

        # Get the merged embeddings
        merged_embedding = self.mvp(text_enc, imgs, self.model)

        # If training include the labels
        return self.model(inputs_embeds=merged_embedding, labels=labels)

In [10]:
class MultiFrameDataset(Dataset):

    def __init__(self, input_file, tokenizer, transform=None):
        with open(input_file) as f:
            self.data = json.load(f)

        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the question and answer at the idx
        qa, img_path = self.data[idx]
        img_path = [os.path.join('DriveLM', p)
                    for p in list(img_path.values())]

        q_text, a_text = qa['Q'], qa['A']
        q_text = f"Question: {q_text} Answer:"

        # Concatenate images into a single tensor
        imgs = [self.transform(read_image(p).float()).to(device) for p in img_path]
        imgs = torch.stack(imgs, dim=0)

        return q_text, imgs, a_text

    def collate_fn(self, batch):

        q_texts, imgs, a_texts = zip(*batch)
        imgs = torch.stack(list(imgs), dim=0)

        encodings = self.tokenizer(q_texts, padding=True, return_tensors="pt").input_ids.to(device)
        labels = self.tokenizer(a_texts, padding=True, return_tensors='pt').input_ids.to(device)

        return encodings, imgs, labels

In [11]:
Config = namedtuple('Instance', ['batch_size', 'learning_rate',
                                 'weight_decay', 'num_workers',
                                 'epochs', 'custom_train', 'gpa_hidden_size',
                                 'lora', 'lora_dim', 'lora_alpha', 'lora_dropout',
                                 'load_checkpoint', 'file_checkpoint', 'checkpoint_frequency',
                                 'freeze_lm', 'lm'])

# We recommend keep all hyperparameters the same besides the following
# file_checkpoint -> Checkpoint folder stored in multi_frame_results folder
# load_checkpoint -> Use if you want to resume training from a checkpoint stored in multi_frame_results
config = Config(
    batch_size = 4,
    learning_rate = 1e-4,
    weight_decay = 0.05,
    num_workers = 0,
    epochs = 6,
    custom_train = True,
    gpa_hidden_size = 128,
    lora = False,
    lora_dim = 64,
    lora_alpha = 32,
    lora_dropout = 0.05,
    load_checkpoint = False,
    file_checkpoint = '20240301-053312',
    checkpoint_frequency = 15000,
    freeze_lm = True,
    lm = 'T5'
)

In [15]:
%cd '/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD/'

/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# /content/drive/MyDrive/ColabNotebooks/EM-VLM4AD/DriveLM

def save_model(model, model_name):
    # Save the model into the designated folder
    path = os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'multi_frame_results', timestr, model_name + '.pth')
    torch.save(model, path)


def val_model(dloader, val_model):
    val_model.eval()
    val_loss = 0

    for idx, (inputs, imgs, labels) in tqdm(enumerate(dloader), total=len(dloader)):
        outputs = val_model(inputs, imgs, labels)
        val_loss += outputs.loss.item()

    return val_loss / len(val_dataloader)


def save_stats(train_loss, val_loss, epochs, lr):
    stats_dict = {
        'losses': losses,
        'val losses': val_losses,
        'min train loss': train_loss,
        'min val loss': val_loss,
        'epochs': epochs,
        'learning rate': lr,
        'LM': 'T5-Base',
        'Image Embedding': 'Patch'
    }

    # Save stats into checkpoint
    with open(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'multi_frame_results', timestr, 'stats.json'), 'w') as f:
        json.dump(stats_dict, f)


def plot_loss(training_loss, val_loss):
    num_epochs = len(training_loss)

    plt.plot(range(1, num_epochs + 1), training_loss, label='Training Loss')
    plt.plot(range(1, num_epochs + 1), val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Num epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'multi_frame_results', timestr, 'loss.png'))


def custom_train(train_loss, val_loss, best_model, epochs, learning_rate):

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)

    for epoch in range(epochs, config.epochs):
        print('-------------------- EPOCH ' + str(epoch) + ' ---------------------')
        model.train()
        epoch_loss = 0

        for step, (inputs, imgs, labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

            # print(inputs.shape, imgs.shape, labels.shape)

            # Forward pass through model
            outputs = model(inputs, imgs, labels)

            # Calculate loss
            loss = outputs.loss
            epoch_loss += loss.item()

            if step % config.checkpoint_frequency == 0:
              print()
              print('Loss: ' + str(loss.item()))

              # Get the hidden states (output)
              hidden_states = outputs.logits

              # Perform decoding (e.g., greedy decoding)
              outputs = torch.argmax(hidden_states, dim=-1)

              text_outputs = [processor.decode(output.to('cpu'), skip_special_tokens=True) for output in outputs]
              text_questions = [processor.decode(q.to('cpu'), skip_special_tokens=True) for q in inputs]
              text_labels = [processor.decode(a.to('cpu'), skip_special_tokens=True) for a in labels]
              print()
              print('Questions:')
              print(text_questions)
              print()
              print('Generated Answers:')
              print(text_outputs)
              print()
              print('Ground Truth Answers:')
              print(text_labels)

            # Back-propogate
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Get train and val loss per batch
        epoch_train_loss = epoch_loss / len(train_dataloader)
        losses.append(epoch_train_loss)

        epoch_val_loss = val_model(val_dataloader, model)
        val_losses.append(epoch_val_loss)

        if not val_loss or min(epoch_val_loss, val_loss) == epoch_val_loss:
            val_loss = epoch_val_loss
            best_model = deepcopy(model.state_dict())
        if not train_loss or min(train_loss, epoch_train_loss) == epoch_train_loss:
            train_loss = epoch_train_loss

        # Adjust learning rate scheduler
        scheduler.step()

        print('Training Loss: ' + str(epoch_train_loss))
        print('Validation Loss: ' + str(epoch_val_loss))
        print('---------------------------------------------')

        # Save model and stats for checkpoints
        save_model(best_model, 'latest_model')
        epochs += 1
        save_stats(train_loss, val_loss, epochs, scheduler.get_last_lr()[0])

    # Save the model and plot the loss
    plot_loss(losses, val_losses)
    return train_loss, val_loss


def train():
    training_config = TrainingArguments(
        output_dir="agopalkr/EfficientDriveLM",
        learning_rate=config.learning_rate,
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=config.batch_size,
        num_train_epochs=config.epochs,
        weight_decay=config.weight_decay,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        config=training_config,
        train_dataset=train_dset,
        eval_dataset=val_dset,
    )

    trainer.train()
    model.push_to_hub("agopalkr/EfficientDriveLM")


def save_experiment(statistics):
    """
    Saves the experiment multi_frame_results to a csv
    :param config: The hyperparameters used
    :param statistics: The accuracies for the training, validation, and test sets
    """
    trial_dict = {
        'Model name': [timestr],
        'Learning rate': [config.learning_rate],
        'Weight decay': [config.weight_decay],
        'Batch size': [config.batch_size],
        'Epochs': [config.epochs],
        'LoRA finetuning': [config.lora],
        'GPA Hidden Size': [config.gpa_hidden_size],
        'LoRA Dimension': [config.lora_dim],
        'LoRA Alpha': [config.lora_alpha],
        'LoRA Dropout': [config.lora_dropout],
        'Freeze T5': [config.freeze_lm],
        'Min Training Loss': [statistics[0]],
        'Min Validation Loss': [statistics[1]],
        'Min Testing Loss': [statistics[2]],
    }

    trial_dict = pd.DataFrame(trial_dict)
    trial_dict.to_csv(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'multi_frame_results', timestr, 'multi_frame_results.csv'), index=False, header=True)


if __name__ == '__main__':

    timestr = time.strftime("%Y%m%d-%H%M%S")

    losses = []
    val_losses = []
    min_train_loss = None
    min_val_loss = None
    best_model = None
    epochs_ran = 0

    # Load processors and models
    model = DriveVLM(config)
    model.to(device)
    print('Trainable Parameters for full model')
    print_trainable_parameters(model)

    if config.lm == 'T5':
      processor = T5Tokenizer.from_pretrained('google-t5/t5-base')
      processor.add_tokens('<')
    else:
      processor = GPT2TokenizerFast.from_pretrained('gpt2-medium')
      processor.pad_token = processor.eos_token

    train_dset = MultiFrameDataset(
        input_file=os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'data', 'multi_frame',
                                'multi_frame_train.json'),
        tokenizer = processor,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
        ])
    )
    val_dset = MultiFrameDataset(
        input_file=os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'data', 'multi_frame',
                                'multi_frame_val.json'),
        tokenizer = processor,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
        ])
    )
    test_dset = MultiFrameDataset(
        input_file=os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'data', 'multi_frame',
                                'multi_frame_test.json'),
        tokenizer = processor,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
        ])
    )

    # Create Dataloaders
    train_dataloader = DataLoader(train_dset, shuffle=True, batch_size=config.batch_size,
                                  num_workers=config.num_workers, collate_fn=train_dset.collate_fn)
    val_dataloader = DataLoader(val_dset, shuffle=True, batch_size=config.batch_size,
                                num_workers=config.num_workers, collate_fn=train_dset.collate_fn)
    test_dataloader = DataLoader(test_dset, shuffle=True, batch_size=config.batch_size,
                                 num_workers=config.num_workers, collate_fn=train_dset.collate_fn)

    if config.custom_train:

        # Load checkpoint if neccesary:
        if config.load_checkpoint:

            print('Loading model from ' + config.file_checkpoint)

            # Load the model and stats from the checkpoint
            model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD',
                                                          'DriveLM',
                                                          'multi_frame_results',
                                                          config.file_checkpoint,
                                                          'latest_model.pth')))
            best_model = DriveVLM(config)
            best_model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD',
                                                               'DriveLM',
                                                               'multi_frame_results',
                                                               config.file_checkpoint,
                                                               'latest_model.pth')))

            with open(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD',
                                   'DriveLM',
                                   'multi_frame_results',
                                   config.file_checkpoint,
                                   'stats.json'), 'r') as f:
                stats = json.load(f)

            min_train_loss, min_val_loss, losses, val_losses, epochs_ran = stats['min train loss'], stats['min val loss'], stats['losses'], stats['val losses'], stats['epochs']

            # Just to correct previous mistake
            # val_losses[:2] = [val_loss*len(val_dataloader_4) for val_loss in val_losses[:2]]
            # val_losses[2:] = [val_loss*len(val_dataloader_2) for val_loss in val_losses[2:]]
            # min_val_loss *= len(val_dataloader_2)

            print(f'Minimum Training Loss: {min_train_loss}')
            print(f'Training Losses: {losses}')
            print(f'Minimum Validation Loss: {min_val_loss}')
            print(f'Validation Losses: {val_losses}')
            print(f'Epochs ran: {epochs_ran}')
            timestr = config.file_checkpoint
        else:
            os.mkdir(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'multi_frame_results', timestr))

        if config.load_checkpoint:
          lr = stats['learning rate']
        else:
          lr = config.learning_rate

        min_train_loss, min_val_loss = custom_train(min_train_loss, min_val_loss, best_model, epochs_ran, lr)
        best_model = DriveVLM(config)
        best_model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive/ColabNotebooks/EM-VLM4AD', 'DriveLM', 'multi_frame_results', timestr, 'latest_model.pth')))
        best_model.to(device)
        test_loss = val_model(test_dataloader, best_model)
        statistics = [min_train_loss, min_val_loss, test_loss]
        save_experiment(statistics)
    else:
        train()

Trainable Parameters for LM model:
Trainable params: 0 || all params: 222903552 || trainable%: 0.0
Trainable Parameters for full model
Trainable params: 9635457 || all params: 320763241 || trainable%: 3.0039155889436846


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


-------------------- EPOCH 0 ---------------------


  0%|          | 0/85346 [00:00<?, ?it/s]


Loss: 7.6028618812561035

Questions:
['Question: Which object is most likely to be occluded by < c2,CAM_FRONT,540.8,487.5>? Would this object affect the ego vehicle? Based on this object, what action of the ego vehicle is dangerous? Answer:', 'Question: What object would consider < c1,CAM_FRONT_RIGHT,371.5,478.4> to be most relevant to its decision? Answer:', 'Question: What object would consider < c3,CAM_FRONT,295.8,535.0> to be most relevant to its decision? Answer:', 'Question: What are objects to the front right of the ego car? Answer:']

Generated Answers:
['of not  no,  No No No No  No No', 'Greatnewest of.ing', 'FalX of is The The The The The The The The', 'is no more on the front left of the ego car  Question']

Ground Truth Answers:
['None, no, none.', 'The ego vehicle.', 'The ego vehicle.', 'There is one car to the front right of the ego car.']


  0%|          | 44/85346 [17:37<569:24:12, 24.03s/it]


KeyboardInterrupt: 